In [29]:
import cv2
import numpy as np

# Loading the object detector model (YOLO)
net = cv2.dnn.readNet('/Users/mac/Documents/R_D/COM_VIS/Project/data/yolov3.weights', '/Users/mac/Documents/R_D/COM_VIS/Project/data/yolov3.cfg')

# Loading the class file (list of class names)
with open('/Users/mac/Documents/R_D/COM_VIS/Project/data/coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Defining the specific classes to count
target_classes = ['truck', 'bus', 'car', 'tanker']

# Opening the video file
cap = cv2.VideoCapture('/Users/mac/Documents/R_D/COM_VIS/Diplome/set/data/scr04.mov')

frame_count = 0
class_counts = {class_name: 0 for class_name in target_classes}
direction_dict = {'right': 0, 'left': 0}

# Dictionary to keep track of counted objects
counted_objects = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 50 != 0:
        continue

    # Cropping the frame to exclude the lower part
    height, width = frame.shape[:2]
    observation_area = frame[:height // 2, :]

    # Preparing the frame for detection
    blob = cv2.dnn.blobFromImage(observation_area, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Getting the detected objects
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outs = net.forward(output_layers)

    # Parsing the detection results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = int(np.argmax(scores))
            confidence = scores[class_id]

            if confidence > 0.5:
                class_name = classes[class_id]
                if class_name in target_classes:
                    # Getting the coordinates of the detection area
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Calculating centroid of the detection area
                    centroid_x = center_x + w // 2
                    centroid_y = center_y + h // 2

                    # Checking if the object has already been counted
                    obj_id = (class_id, centroid_x, centroid_y)
                    if obj_id not in counted_objects:
                        # Incrementing the count for the detected class
                        class_counts[class_name] += 1

                        # Checking direction based on centroid position
                        if centroid_x > width // 2:
                            direction = 'right'
                        else:
                            direction = 'left'

                        # Incrementing direction count
                        direction_dict[direction] += 1

                        # Marking the object as counted
                        counted_objects[obj_id] = True

                        # Drawing rectangle around the detected object
                        cv2.rectangle(observation_area, (center_x - w // 2, center_y - h // 2), 
                                      (center_x + w // 2, center_y + h // 2), (0, 255, 0), 2)
                        # Displaying class name and direction
                        cv2.putText(observation_area, f'{class_name}, {direction}', (center_x, center_y), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Displaying the frame with detected objects
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Closing the video file
cap.release()

# Displaying the non-zero class counts in a table format
print("Class Counts:")
print("--------------")
print("| Class | Quantity | Movement Direction |")
print("--------------------------------------")
for class_name, count in class_counts.items():
    if count > 0:
        print(f"| {class_name} | {count} | {direction_dict['right']} Right, {direction_dict['left']} Left |")
print("--------------------------------------")



Class Counts:
--------------
| Class | Quantity | Movement Direction |
--------------------------------------
| truck | 2 | 15 Right, 8 Left |
| car | 21 | 15 Right, 8 Left |
--------------------------------------
